In [830]:
from PIL import Image
from pyzbar.pyzbar import decode
import cv2
from pyzbar.pyzbar import ZBarSymbol
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import math

In [292]:
# Detect QR code and bar code
#data = decode(Image.open('/Users/garywang/Downloads/IMG_5106.png'))
#print(data)

In [291]:
# Decode the info from qr code
#for i in data:
#    print(i.data.decode("utf-8"))

In [41]:
# Process image into gray or binary image
#image = cv2.imread('/Users/garywang/Downloads/IMG_5107.png')
#gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#_, thresh = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
# Resize image
#image = cv2.resize(thresh, (1600, 1200))

# Show image
#cv2.namedWindow("Images")
#cv2.imshow('Images',thresh)
# Close window
#cv2.waitKey(1)
#cv2.destroyAllWindows()

In [321]:
# Find position of colons to get info from string
def findMultiStr(string):
    colonPos = []
    for i,j in enumerate(string):
        if j == ':':
            colonPos.append(i)
    
    return colonPos

In [1021]:
# Look for just qrcode
from pyzbar.pyzbar import ZBarSymbol
data = decode(Image.open('/Users/garywang/Downloads/test/IMG_5115.png'), symbols=[ZBarSymbol.QRCODE])
print(data)

[Decoded(data=b'**\xe5\x8c\x97\xe6\x97\xa5\xe6\x9c\xac\xe8\x91\xa1\xe8\x90\x84\xe8\xbb\x9f\xe7\xb3\x96:1:41:\xe6\x9e\x9c\xe6\xb1\x81\xe8\xbb\x9f\xe7\xb3\x96\xe7\x99\xbd\xe8\x91\xa1\xe8\x90\x84:1:38:\xe6\x9e\x97\xe9\xb3\xb3\xe7\x87\x9f\xe9\xae\xae\xe4\xb9\xb3:1:35                                                                                           ', type='QRCODE', rect=Rect(left=1529, top=2017, width=276, height=277), polygon=[Point(x=1529, y=2287), Point(x=1800, y=2294), Point(x=1805, y=2024), Point(x=1533, y=2017)]), Decoded(data=b'LF542494401100321509800000132000001400000000028979112fv0FeZeOySjKQBy73+h3nQ==:**********:6:8:1:\xe7\xb9\xbd\xe7\xb4\x9b\xe6\xa8\x82\xe5\xb7\xa7\xe5\x85\x8b\xe5\x8a\x9b:1:79:\xe5\xb0\x8f\xe7\xa3\xa8\xe5\x9d\x8a\xe6\xa4\xb0\xe6\xbc\xbf:1:55:\xe4\xb8\x89\xe5\xbe\x97\xe5\x88\xa9\xe9\xa0\x82\xe7\xb4\x9a\xe5\x95\xa4\xe9\x85\x92:1:42:', type='QRCODE', rect=Rect(left=1173, top=2012, width=271, height=274), polygon=[Point(x=1173, y=2279), Point(x=1440, y=2286)

In [1022]:
# Decode
decodeData = []
for i in data:
    decodeData.append(i.data.decode("utf-8"))
    print(i.data.decode("utf-8"))

**北日本葡萄軟糖:1:41:果汁軟糖白葡萄:1:38:林鳳營鮮乳:1:35                                                                                           
LF542494401100321509800000132000001400000000028979112fv0FeZeOySjKQBy73+h3nQ==:**********:6:8:1:繽紛樂巧克力:1:79:小磨坊椰漿:1:55:三得利頂級啤酒:1:42:


In [552]:
# Read companyList and drop some columns

companyList = pd.read_csv('/Users/garywang/Downloads/BGMOPEN1.csv')
companyList = companyList.drop([0])
companyList = companyList.drop(['行業代號1','名稱1','行業代號2','名稱2','行業代號3','名稱3'],axis=1)
companyList['統一編號'] = companyList['統一編號'].astype(int)
companyList['行業代號'] = companyList['行業代號'].astype(int)
companyList['資本額'] = companyList['資本額'].astype(int)
#companyList['總機構統一編號'] = companyList['總機構統一編號'].astype(int)
companyList['設立日期'] = companyList['設立日期'].astype(int)

#for l in range(1,len(companyList)):
#    if pd.isnull(companyList.loc[l,'設立日期']) == False:
#        companyList.loc[l,'設立日期'] = int(companyList.loc[l,'設立日期'])+19110000
#        companyList.loc[l,'設立日期'] = pd.to_datetime(str(companyList.loc[l,'設立日期']), format='%Y-%m-%d').date()
#        companyList.loc[l,'設立日期'] = companyList.loc[l,'設立日期'].strftime('%d-%b-%y')

companyList.to_csv('/Users/garywang/Downloads/BGMOPEN1.csv')

In [1015]:
# Complete code
#receiptDataFrame columns = ['receiptNum, date, expense, taxIdNumOfBuyer, taxIdNumOfSeller, totalNumOfProduct, product, number, price'])
receiptDataFrame = pd.DataFrame()
k = 0

# path of Invoice's file
path = '/Users/garywang/Downloads/test'
colonPos = []

# Read all of the png's image from the file of invoice
allList = os.walk(path, 'w')
for dirPath, dirNames, files in allList:
    for m in files:
        # Find qr code from image
        data = decode(Image.open(dirPath + '/' + m), symbols=[ZBarSymbol.QRCODE])
        
        decodeData = []
        for n in data:
            # Decode qr code 
            decodeData.append(n.data.decode("utf-8"))
        
        if decodeData == []:
            k -= 1
        
        # Fill information of invoice into dataframe
        a = 0
        b = 0
        for i in decodeData:
            # get info from first sentence
            if i[0] != '*':
                
                b += 1
                receiptDataFrame.loc[k,'receiptNum'] = i[0:10]
                receiptDataFrame.loc[k,'date'] = i[10:17]
                receiptDataFrame.loc[k,'expense'] = i[29:37]
                receiptDataFrame.loc[k,'taxIdNumOfBuyer'] = i[37:45]
                receiptDataFrame.loc[k,'taxIdNumOfSeller'] = i[45:53]
                
                # call function findMultStr
                colonPos = findMultiStr(i)
                receiptDataFrame.loc[k,'totalNumOfProduct'] = i[colonPos[2]+1 : colonPos[3]]
    
                if len(colonPos) > 5: 
                    if a == 0:
                        for l in range(min(int((len(colonPos)-5)/2),2)):
                            receiptDataFrame.loc[k,'product'] = i[colonPos[4+l*3]+1:colonPos[5+l*3]] 
                            receiptDataFrame.loc[k,'number'] = i[colonPos[5+l*3]+1:colonPos[6+l*3]] 
                            receiptDataFrame.loc[k,'price'] = i[colonPos[6+l*3]+1:] 
                    elif a == 1:
                        for l in range(int((len(colonPos)+1-5)/3)):
                            receiptDataFrame.loc[k,'product'] = receiptDataFrame.loc[k,'product'] +','+ i[colonPos[4+l*3]+1:colonPos[5+l*3]] 
                            receiptDataFrame.loc[k,'number'] = receiptDataFrame.loc[k,'number'] +','+ i[colonPos[5+l*3]+1:colonPos[6+l*3]] 
                            
                            if l != int((len(colonPos)-5)/3)-1:
                                receiptDataFrame.loc[k,'price'] = receiptDataFrame.loc[k,'price'] +','+ i[colonPos[6+l*3]+1:colonPos[6+l*3+1]] 
                            elif l == int((len(colonPos)-5)/3)-1:
                                receiptDataFrame.loc[k,'price'] = receiptDataFrame.loc[k,'price'] +','+ i[colonPos[6+l*3]+1:]
    
            # get info from second sentence
            if i[0] == '*':
                colonPos = findMultiStr(i)
                
                # Remove '\u3000' from string
                i = i.replace('\u3000','')
                
                if len(colonPos) > 0:
                    a += 1
                    #k -= 1
                    for j in range(int((len(colonPos)+1)/3)): 
                        if j == 0:
                            
                            
                            
                            if b == 1 and pd.isnull(receiptDataFrame.loc[k,'product']) == False:
                                receiptDataFrame.loc[k,'product'] = receiptDataFrame.loc[k,'product'] +','+ i[2 : colonPos[0]] 
                                receiptDataFrame.loc[k,'number'] = receiptDataFrame.loc[k,'number'] +','+ i[colonPos[0]+1:colonPos[1]]
                            
                            # Check lenth of colonPos which is equal than 2 or not and adjust 
                            # Otherwise it will show error
                                if len(colonPos) == 2:
                                    receiptDataFrame.loc[k,'price'] = receiptDataFrame.loc[k,'price'] +','+ i[colonPos[1]+1:len(i)] 
                                if len(colonPos) > 2:
                                    receiptDataFrame.loc[k,'price'] = receiptDataFrame.loc[k,'price'] +','+ i[colonPos[1]+1:colonPos[2]] 
                            
                            if b == 0 or pd.isnull(receiptDataFrame.loc[k,'product']) == True:
                            ##****Problem*****
                                receiptDataFrame.loc[k,'product'] = i[2 : colonPos[0]] 
                                receiptDataFrame.loc[k,'number'] = i[colonPos[0]+1:colonPos[1]]
                            
                            # Check lenth of colonPos which is equal than 2 or not and adjust 
                            # Otherwise it will show error
                                if len(colonPos) == 2:
                                    receiptDataFrame.loc[k,'price'] = i[colonPos[1]+1:len(i)] 
                                if len(colonPos) > 2:
                                    receiptDataFrame.loc[k,'price'] = i[colonPos[1]+1:colonPos[2]] 
                            
                            
                        if j > 0:
                            receiptDataFrame.loc[k,'product'] = receiptDataFrame.loc[k,'product'] + ',' + i[colonPos[j*3-1]+1 : colonPos[j*3]] 
                            receiptDataFrame.loc[k,'number'] = receiptDataFrame.loc[k,'number'] + ',' + i[colonPos[j*3]+1:colonPos[j*3+1]] 
                            
                            if j < int((len(colonPos)+1)/3)-1:
                                receiptDataFrame.loc[k,'price'] = receiptDataFrame.loc[k,'price'] + ',' + i[colonPos[j*3+1]+1:colonPos[j*3+2]] 
                            else:
                                receiptDataFrame.loc[k,'price'] = receiptDataFrame.loc[k,'price'] + ',' + i[colonPos[j*3+1]+1:]
        k += 1 # Move to next row

# Convert expense(hex) into int(Dec)
for l in range(len(receiptDataFrame)):
    if pd.isnull(receiptDataFrame.loc[l,'expense']) == False:
        receiptDataFrame.loc[l,'expense'] = int(receiptDataFrame.loc[l,'expense'],16)

# Convert date into datatime form
for l in range(len(receiptDataFrame)):
    if pd.isnull(receiptDataFrame.loc[l,'date']) == False:
        receiptDataFrame.loc[l,'date'] = int(receiptDataFrame.loc[l,'date'])+19110000
        receiptDataFrame.loc[l,'date'] = pd.to_datetime(str(receiptDataFrame.loc[l,'date']), format='%Y-%m-%d').date()
        receiptDataFrame.loc[l,'date'] = receiptDataFrame.loc[l,'date'].strftime('%d-%b-%y')

# Add new "city" and "address" columns by searching from companyList
companyList = pd.read_csv('/Users/garywang/Downloads/BGMOPEN1.csv')
for l in range(len(receiptDataFrame)):
    if pd.isnull(receiptDataFrame.loc[l,'taxIdNumOfSeller']) == False:
        address = companyList[companyList['統一編號'] == int(receiptDataFrame.loc[l,'taxIdNumOfSeller'])].iloc[0,1]
        receiptDataFrame.loc[l,'SellerCity'] = address[0:6]
        
# Remove colon and space from price
for l in range(len(receiptDataFrame)):
    receiptDataFrame.loc[l,'price'] = receiptDataFrame.loc[l,'price'].replace(':','')
    receiptDataFrame.loc[l,'price'] = receiptDataFrame.loc[l,'price'].replace(' ','')
    

In [1016]:
receiptDataFrame

,product,number,price,receiptNum,date,expense,taxIdNumOfBuyer,taxIdNumOfSeller,totalNumOfProduct,SellerCity
0,"代收當筆-交貨便寄件袋1元,代收當筆-UD抽衛2包20元,代收當筆-王子麵3件20元,手續費...","2,198,198,1,198","0,0,0,4,0",LT43624874,10-Apr-21,4,00000000,54915924,5,新北市三重區
1,涼拌飯(豬),1,180.0,LY47754818,16-Mar-21,180,00000000,54911841,1,臺北市信義區
2,"小磨坊椰漿,瑞穗全脂鮮乳,限時特價,WMF印花１張,卡麥隆黑麥檸檬,林鳳營蜂蜜優格,光州辣炒年糕","1,1,1,2,1,2,1","55,42,30,0,149,139,79",LF52339750,28-Mar-21,509,00000000,24804767,7,新北市三重區
3,邁｡蝟ｮ鮟槫次蜻ｳ蜆ｪ驟ｪ荵ｳ,1,59,MC58067220,14-Apr-21,59,00000000,70785378,1,新北市三重區
4,一配肉鬆起酥麵包,1,25,LM31015428,22-Apr-21,18,00000000,83422667,1,新北市三重區
5,"(區)經典布丁奶酪8杯裝(鮮奶),KT閨ｯ蜷崎ｲｼ邏咎ｻ樊丙","1,5","299,0",LT06118303,14-Apr-21,299,00000000,27950313,2,臺南市歸仁區
6,餐食,1,130,LF06276574,30-Mar-21,130,00000000,16095410,1,臺北市信義區
7,耆盛西谷米200g,1,28,MC58045418,22-Mar-21,28,00000000,70785378,1,新北市三重區
8,舊拍3本500元(紅,1,500,ME75301957,06-Mar-21,500,00000000,53519968,1,臺北市中正區
9,",145","牛燒肉,半熟玉子","1,115",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [568]:
companyList.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1515908 entries, 1 to 1515908
Data columns (total 10 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   營業地址     1515908 non-null  object 
 1   統一編號     1515908 non-null  int64  
 2   總機構統一編號  71245 non-null    float64
 3   營業人名稱    1515908 non-null  object 
 4   資本額      1515908 non-null  int64  
 5   設立日期     1515908 non-null  int64  
 6   組織別名稱    1515908 non-null  object 
 7   使用統一發票   1515908 non-null  object 
 8   行業代號     1515908 non-null  int64  
 9   名稱       1515863 non-null  object 
dtypes: float64(1), int64(4), object(5)
memory usage: 127.2+ MB
